In [0]:
!pip install eli5

In [0]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance

from ast import literal_eval
from tqdm import tqdm_notebook #wizualizacja postępów pracy

In [4]:
cd "/content/drive/My Drive/Colab Notebooks/dw_matrix"

/content/drive/My Drive/Colab Notebooks/dw_matrix


In [0]:
df = pd.read_csv('data/men_shoes.csv', low_memory=False)

In [0]:
def run_model(feats, model = DecisionTreeRegressor(max_depth=5)):
  X = df[ feats ].values
  y = df['prices_amountmin' ].values

  scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [7]:
df['brand_cat'] = df['brand'].map(lambda x: str(x).lower()).factorize()[0]
run_model(['brand_cat'])

(-58.133398968282776, 4.206122611474276)

In [8]:
#usprawniamy wyniki zmieniając model

model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
run_model(['brand_cat'], model)

(-57.31783843165656, 4.181246596160967)

In [0]:
df.head()

In [10]:
test = {'key': 'value'}
test['key']

str(test)

"{'key': 'value'}"

In [0]:
#parsowanie kolumny features
def parse_features(x):
  output_dict = {}
  if str(x) == 'nan' : return output_dict

  features = literal_eval(x.replace('\\"', '"'))
  for item in features:
    key = item['key'].lower().strip()
    value = item['value'][0].lower().strip()

    output_dict[key] = value
  return output_dict

df['features_parsed'] = df['features'].map(parse_features)

In [12]:
keys = set()

df['features_parsed'].map( lambda x: keys.update(x.keys()))

len(keys)

476

In [13]:
def get_name_feat(key):
  return 'feat_' + key

for key in tqdm_notebook(keys):
  df[get_name_feat(key)] = df.features_parsed.map(lambda feats: feats[key] if key in feats else np.nan)

In [14]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension',
       ...
       'feat_is energy star-certified', 'feat_uv protection', 'feat_shoe size',
       'feat_seller inventory number', 'feat_toe type', 'feat_transactionid',
       'feat_series name', 'feat_band material', 'feat_stability',
       'feat_waist size'],
      dtype='object', length=526)

In [15]:
#szukamy cechy która ma jakąś wartość w %
df [ False == df['feat_athlete'].isnull() ].shape[0] / df.shape[0] * 100

0.0437636761487965

In [0]:
keys_stat = {}
for key in keys:
  keys_stat[key] = df [ False == df[get_name_feat(key)].isnull() ].shape[0] / df.shape[0] * 100

In [0]:
{k:v for k,v in keys_stat.items() if v > 30}

In [0]:
df['feat_brand_cat'] = df['feat_brand'].factorize()[0]
df['feat_color_cat'] = df['feat_color'].factorize()[0]
df['feat_gender_cat'] = df['feat_gender'].factorize()[0]
df['feat_manufacturer part number_cat'] = df['feat_manufacturer part number'].factorize()[0]
df['feat_material_cat'] = df['feat_material'].factorize()[0]

df['feat_sport_cat'] = df['feat_sport'].factorize()[0]
df['feat_style_cat'] = df['feat_style'].factorize()[0]

for key in keys:
  df[get_name_feat(key) + '_cat'] = df[get_name_feat(key)].factorize()[0]

In [0]:
df [ df.brand != df.feat_brand ][ ['brand', 'feat_brand'] ].head() #nieznormalizowane

In [0]:
df['brand'] = df['brand'].map(lambda x: str(x).lower())
df [ df.brand != df.feat_brand ][ ['brand', 'feat_brand'] ].head() 

In [0]:
feats = ['']

In [22]:
model = RandomForestRegressor(max_depth=5, n_estimators=100)
run_model(['brand_cat'], model)

(-57.31220804849092, 4.186531317492023)

In [0]:
feats_cat = [x for x in df.columns if 'cat' in x]
feats_cat

In [78]:
feats = ['brand_cat', 'feat_brand_cat', 'feat_gender_cat', 'feat_material_cat', 'feat_style_cat', 'feat_condition_cat', 'feat_weight_cat']
feats += []

model = RandomForestRegressor(max_depth=5, n_estimators=100)
result = run_model(feats, model)
print(result)

(-56.932106951265915, 4.265238005236932)


In [77]:
X = df[ feats ].values
y = df['prices_amountmin'].values

m = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
m.fit(X, y)

print(result)
perm = PermutationImportance(m, random_state=1).fit(X, y);
eli5.show_weights(perm, feature_names=feats)

(-56.959514034666554, 4.325792094284986)


Weight,Feature
0.2569 ± 0.0100,brand_cat
0.1002 ± 0.0073,feat_material_cat
0.0429 ± 0.0022,feat_gender_cat
0.0200 ± 0.0006,feat_brand_cat
0.0131 ± 0.0008,feat_weight_cat
0.0053 ± 0.0009,feat_style_cat
0.0021 ± 0.0003,feat_condition_cat


In [0]:
df['brand'].value_counts(normalize=True)

In [0]:
df [ df['brand'] == 'nike'].features_parsed.sample(5).values

In [0]:
!git add matrix_one/day5.ipynb